# Classificação de Redes Complexas

Amanda Araujo Silva - nº USP: 10260441

Disciplina: Processos Dinâmicos em Redes Complexas (ICMC-USP)

Prof. Francisco A. Rodrigues

## Descrição do trabalho

Selecione 3 redes biológicas, 3 redes sociais e 3 redes tecnológicas desses endereços:

* https://networks.skewed.de
* http://konect.cc/networks/
* https://icon.colorado.edu/


Faça a classificação das redes usando os modelos e medidas que aprendemos na aula.
Não se esqueça de selecionar o mesmo N e grau médio que a rede original na construção
dos modelos.
Verifique qual o modelo mais adequado para cada rede.

> **Hipótese: redes do mesmo tipo seguem o mesmo modelo.**

Verifique se essa hipótese é verdadeira.

## Bibliotecas e arquivos

In [15]:
# Bibliotecas:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import pearsonr
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

### Graph-tool library

In [2]:
!wget https://downloads.skewed.de/skewed-keyring/skewed-keyring_1.0_all_$(lsb_release -s -c).deb
!dpkg -i skewed-keyring_1.0_all_$(lsb_release -s -c).deb
!echo "deb [signed-by=/usr/share/keyrings/skewed-keyring.gpg] https://downloads.skewed.de/apt $(lsb_release -s -c) main" > /etc/apt/sources.list.d/skewed.list
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

--2024-07-01 15:02:03--  https://downloads.skewed.de/skewed-keyring/skewed-keyring_1.0_all_jammy.deb
Resolving downloads.skewed.de (downloads.skewed.de)... 49.12.93.194
Connecting to downloads.skewed.de (downloads.skewed.de)|49.12.93.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23192 (23K) [application/x-debian-package]
Saving to: ‘skewed-keyring_1.0_all_jammy.deb’

skewed-keyring_1.0_ 100%[===================>]  22.65K  --.-KB/s    in 0.1s    

2024-07-01 15:02:04 (220 KB/s) - ‘skewed-keyring_1.0_all_jammy.deb’ saved [23192/23192]

Selecting previously unselected package skewed-keyring.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack skewed-keyring_1.0_all_jammy.deb ...
Unpacking skewed-keyring (1.0) ...
Setting up skewed-keyring (1.0) ...
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https:

In [3]:
# Colab uses a Python install that deviates from the system's! Bad colab! We need some workarounds.
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  python3-cairo*
0 upgraded, 0 newly installed, 1 to remove and 48 not upgraded.
After this operation, 310 kB disk space will be freed.
(Reading database ... 130326 files and directories currently installed.)
Removing python3-cairo:amd64 (1.20.1-3build1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev libglib2.0-dev libglib2.0-dev-bin
  libice-dev liblzo2-2 libmount-dev libmount1 libpixman-1-dev libselinux1-dev libsepol-dev
  libsm

Now we can use graph-tool as any other Python module



In [4]:
from graph_tool.all import *
import graph_tool as gt

## Leitura das redes

Obtenção das redes em https://networks.skewed.de

Importação da rede diretamente pela biblioteca Graph-tool, seguida de transformação para networkx.

In [5]:
def graph_tool_to_networkx(graph_tool_graph):
    nx_graph = nx.Graph()
    for v in graph_tool_graph.vertices():
        attrs = {k: graph_tool_graph.vp[k][v] for k in graph_tool_graph.vp.keys()}
        nx_graph.add_node(int(v), **attrs)
    for e in graph_tool_graph.edges():
        source = int(e.source())
        target = int(e.target())
        attrs = {k: graph_tool_graph.ep[k][e] for k in graph_tool_graph.ep.keys()}
        nx_graph.add_edge(source, target, **attrs)
    return nx_graph

In [6]:
# Leitura dos dados: 9 redes
grafos = []
grafos.append(graph_tool_to_networkx(gt.collection.ns["malaria_genes/HVR_1"])) #biológica: https://networks.skewed.de/net/malaria_genes
grafos.append(graph_tool_to_networkx(gt.collection.ns["celegans_metabolic"]))  #biológica: https://networks.skewed.de/net/celegans_metabolic
grafos.append(graph_tool_to_networkx(gt.collection.ns["yeast_transcription"])) #biológica: https://networks.skewed.de/net/yeast_transcription
grafos.append(graph_tool_to_networkx(gt.collection.ns["lesmis"]))             #social: https://networks.skewed.de/net/lesmis
grafos.append(graph_tool_to_networkx(gt.collection.ns["facebook_friends"]))   #social: https://networks.skewed.de/net/facebook_friends
grafos.append(graph_tool_to_networkx(gt.collection.ns["jazz_collab"]))        #social: https://networks.skewed.de/net/jazz_collab
grafos.append(graph_tool_to_networkx(gt.collection.ns["route_views/19971108"])) #tecnológica: https://networks.skewed.de/net/route_views
grafos.append(graph_tool_to_networkx(gt.collection.ns["power"]))             #tecnológica: https://networks.skewed.de/net/power
grafos.append(graph_tool_to_networkx(gt.collection.ns["internet_top_pop/Kdl"])) #tecnológica: https://networks.skewed.de/net/internet_top_pop

In [7]:
redes = ['malaria_genes', 'celegans_metabolic', 'yeast_transcription', 'lesmis', 'facebook_friends', 'jazz_collab', 'route_views', 'power', 'internet_top_pop_kdl']

## Informação das redes

In [8]:
# Informações das redes
for index, G in enumerate(grafos):
  if index == 0: print('Redes biológicas')
  elif index == 3: print('Redes sociais')
  elif index == 6: print('Redes tecnológicas')
  print(redes[index])
  print("Nodes:", nx.number_of_nodes(G))
  print("Edges:", nx.number_of_edges(G))
  print("-----------------------------")

Redes biológicas
malaria_genes
Nodes: 307
Edges: 2812
-----------------------------
celegans_metabolic
Nodes: 453
Edges: 2040
-----------------------------
yeast_transcription
Nodes: 916
Edges: 1093
-----------------------------
Redes sociais
lesmis
Nodes: 77
Edges: 254
-----------------------------
facebook_friends
Nodes: 362
Edges: 1988
-----------------------------
jazz_collab
Nodes: 198
Edges: 2742
-----------------------------
Redes tecnológicas
route_views
Nodes: 3015
Edges: 5539
-----------------------------
power
Nodes: 4941
Edges: 6594
-----------------------------
internet_top_pop_kdl
Nodes: 754
Edges: 895
-----------------------------


In [9]:
# Número de componentes conectados
for index, G in enumerate(grafos):
  print(str(index) + ': ' + str(nx.number_connected_components(G))) #2, 4

0: 1
1: 1
2: 237
3: 1
4: 20
5: 1
6: 1
7: 1
8: 1


## Preparação das redes

In [10]:
# Preparação da rede
for i in range(len(grafos)):
  G = grafos[i]
  G = G.to_undirected()                     # não direcionada
  G.remove_edges_from(nx.selfloop_edges(G)) # remoção auto-loops
  Gcc = sorted(nx.connected_components(G), key = len, reverse = True) # maior componente conectado
  G = G.subgraph(Gcc[0])
  G = nx.convert_node_labels_to_integers(G, first_label = 0)
  grafos[i] = G

In [11]:
# Número de componentes conectados
for index, G in enumerate(grafos):
  print(str(index) + ': ' + str(nx.number_connected_components(G)))

0: 1
1: 1
2: 1
3: 1
4: 1
5: 1
6: 1
7: 1
8: 1


## Medidas de rede

In [12]:
# Medidas de rede
def measures(G):

  # graus
  vk = dict(G.degree())
  vk = list(vk.values())
  vk = np.array(vk)

  # distribuição momento de graus
  def momment_of_degree_distribution(G, m):
      M = 0
      N = len(G)
      for i in G.nodes:
          M = M + G.degree(i)**m
      M = M/N
      return M
  k1 = momment_of_degree_distribution(G, 1)
  k2 = momment_of_degree_distribution(G, 2)

  # variância
  var = momment_of_degree_distribution(G, 2) - momment_of_degree_distribution(G, 1)**2

  # average clustering
  av_cl = nx.average_clustering(G)

  # menor caminho médio
  l = nx.average_shortest_path_length(G)

  # grau de assortatividade
  r = nx.degree_assortativity_coefficient(G)

  return  k1, k2, var, av_cl, l, r # lista de medidas

**Vetor de características**

In [13]:
vetores_caracteristicos = []
for G in grafos:
  k1, k2, var, av_cl, l, r = measures(G) # Cálculo medidas da rede
  X_net = [k1, k2, var, av_cl, l, r]    # Vetor de características da rede
  print(X_net)
  vetores_caracteristicos.append(X_net)

[18.319218241042346, 457.7785016286645, 122.18474466572587, 0.5701523452772743, 3.5190862447041793, 0.5991478526809376]
[8.940397350993377, 358.49006622516555, 278.55936143151615, 0.646463092156505, 2.6637851882240327, -0.2258208879683295]
[3.2078313253012047, 43.286144578313255, 32.99596276672957, 0.048817590830936314, 5.200239873521234, -0.4083912916261682]
[6.597402597402597, 79.53246753246754, 36.006746500252994, 0.5731367499320134, 2.6411483253588517, -0.16522513442237025]
[11.878419452887538, 260.8632218844985, 119.76637318576141, 0.6052943472285397, 3.584216769219364, 0.07388134601973691]
[27.696969696969695, 1070.2424242424242, 303.1202938475667, 0.6174507021536305, 2.2350407629595446, 0.020237399275047713]
[3.4202321724709783, 314.68059701492535, 302.9826089013198, 0.18154742616600722, 3.761537809734781, -0.22891866576185982]
[2.66909532483303, 10.332726168791742, 3.2086563157462056, 0.08010361108159711, 18.989185424445708, 0.0034569877442048825]
[2.374005305039788, 6.34482758

## Classificação

**Geração das redes**: geração de redes de 3 modelos distintos: ER, WS e BA com respectivos tamanho e grau médio iguais à rede de interesse.

**Classificação**: vetor de características da rede, comparado com o vetor de característica de redes sintéticas ER, WS, BA geradas com o mesmo número de vértices N e grau médio \<k\>.

* K-vizinhos mais próximos (KNN)

In [16]:
for g in range(len(grafos)):

  # Modelos de rede: N, <k> iguais à rede original
  cl = ['ER','WS','BA'] # 0.0, 1.0, 2.0

  ## Criação de redes sinéticas
  X = [] # vetores característicos
  y = [] # modelo da rede
  n_nets = 30
  N = nx.number_of_nodes(grafos[g]) #G

  #ER networks
  av_degree = vetores_caracteristicos[g][0] #k1
  p = av_degree/(N-1)
  for i in range(0, n_nets):
      GER = nx.gnp_random_graph(N, p, seed=None, directed=False)
      Gcc = sorted(nx.connected_components(GER), key=len, reverse=True)
      GER = GER.subgraph(Gcc[0])
      GER = nx.convert_node_labels_to_integers(GER, first_label=0)
      k1, k2, variance, av_cl, l, r = measures(GER)
      x = [k1, k2, variance, av_cl, l, r]
      X.append(x)
      y.append(0.0)

  #WS networks
  k = int(av_degree)
  p = 0.1 #probability of rewiring
  for i in range(0, n_nets):
      GWS = nx.watts_strogatz_graph(N, k, p, seed=None)
      Gcc = sorted(nx.connected_components(GWS), key=len, reverse=True)
      GWS = GWS.subgraph(Gcc[0])
      GWS = nx.convert_node_labels_to_integers(GWS, first_label=0)
      k1, k2, variance, av_cl, l, r = measures(GWS)
      x = [k1, k2, variance, av_cl, l, r]
      X.append(x)
      y.append(1.0)

  # BA networks
  m = int(av_degree/2)
  for i in range(0, n_nets):
      GBA = nx.barabasi_albert_graph(N, m)
      Gcc = sorted(nx.connected_components(GBA), key=len, reverse=True)
      GBA = GBA.subgraph(Gcc[0])
      GBA = nx.convert_node_labels_to_integers(GBA, first_label=0)
      k1,k2,variance,av_cl,l,r = measures(GBA)
      x = [k1,k2,variance,av_cl,l,r]
      X.append(x)
      y.append(2.0)

  X = np.array(X)
  y = np.array(y)

  # Classificação
  scaler = StandardScaler().fit(X)
  X = scaler.transform(X)

  X_net = np.array(vetores_caracteristicos[g])
  X_net = X_net.reshape(1,len(X_net))
  X_net = scaler.transform(X_net)

  k = 5
  model = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
  model.fit(X, y) # predição no conjunto de teste

  y_pred = model.predict(X_net)
  print('Classe grafo', redes[g], ': ', cl[int(y_pred[0])])

Classe grafo malaria_genes :  WS
Classe grafo celegans_metabolic :  BA
Classe grafo yeast_transcription :  ER
Classe grafo lesmis :  BA
Classe grafo facebook_friends :  ER
Classe grafo jazz_collab :  ER
Classe grafo route_views :  ER
Classe grafo power :  ER
Classe grafo internet_top_pop_kdl :  ER


## Resultados

**Resultados Classificação de Redes**

Redes biológicas:
1.   'malaria_genes':  WS
2.   'celegans_metabolic': BA
3.   'yeast_transcription': ER

Redes sociais:
1.   'lesmis': BA
2.   'facebook_friends': ER
3.   'jazz_collab': ER

Redes tecnológicas:
1.   'route_views': ER
2.   'power': ER
3.   'internet_top_pop_kdl': ER




In [24]:
# Vetores característicos WS
print("Medidas:", vetores_caracteristicos[0], "| Rede:", redes[0])

Medidas: [18.319218241042346, 457.7785016286645, 122.18474466572587, 0.5701523452772743, 3.5190862447041793, 0.5991478526809376] | Rede: malaria_genes


In [23]:
# Vetores característicos BA
print("Medidas:", vetores_caracteristicos[1], "| Rede:", redes[1])
print("Medidas:", vetores_caracteristicos[3], "| Rede:", redes[3])

Medidas: [8.940397350993377, 358.49006622516555, 278.55936143151615, 0.646463092156505, 2.6637851882240327, -0.2258208879683295] | Rede: celegans_metabolic
Medidas: [6.597402597402597, 79.53246753246754, 36.006746500252994, 0.5731367499320134, 2.6411483253588517, -0.16522513442237025] | Rede: lesmis


In [25]:
# Vetores característicos ER
print("Medidas:", vetores_caracteristicos[2], "| Rede:", redes[2])
print("Medidas:", vetores_caracteristicos[4], "| Rede:", redes[4])
print("Medidas:", vetores_caracteristicos[5], "| Rede:", redes[5])
print("Medidas:", vetores_caracteristicos[6], "| Rede:", redes[6])
print("Medidas:", vetores_caracteristicos[7], "| Rede:", redes[7])
print("Medidas:", vetores_caracteristicos[8], "| Rede:", redes[8])

Medidas: [3.2078313253012047, 43.286144578313255, 32.99596276672957, 0.048817590830936314, 5.200239873521234, -0.4083912916261682] | Rede: yeast_transcription
Medidas: [11.878419452887538, 260.8632218844985, 119.76637318576141, 0.6052943472285397, 3.584216769219364, 0.07388134601973691] | Rede: facebook_friends
Medidas: [27.696969696969695, 1070.2424242424242, 303.1202938475667, 0.6174507021536305, 2.2350407629595446, 0.020237399275047713] | Rede: jazz_collab
Medidas: [3.4202321724709783, 314.68059701492535, 302.9826089013198, 0.18154742616600722, 3.761537809734781, -0.22891866576185982] | Rede: route_views
Medidas: [2.66909532483303, 10.332726168791742, 3.2086563157462056, 0.08010361108159711, 18.989185424445708, 0.0034569877442048825] | Rede: power
Medidas: [2.374005305039788, 6.344827586206897, 0.7089263978498401, 0.02833775419982316, 22.72654386873373, -0.10462028367002348] | Rede: internet_top_pop_kdl


## Conclusão

**Conclusão**: os resultados obtidos indicam que não haveria evidência experimental para aceitar a hipótese de que redes do mesmo tipo seguem o mesmo modelo de maneira universal. Ou seja, redes de uma mesma natureza - biológica, social ou tecnológica - poderíam vir a seguir modelos de rede distintos, como obsrevado para as redes biológicas, grupo dentro do qual cada rede foi classificada como um modelo diferente. A classificação usando KNN para as redes empregadas mostra que talvez a hipótese se sustente para redes tecnológicas.

### Rascunho: Etapas separadas

In [213]:
g = 2 # grafo

# Modelos de rede: N, <k> iguais à rede original
cl = ['ER','WS','BA'] # 0.0, 1.0, 2.0

## Criação de redes sinéticas
X = [] # vetores característicos
y = [] # modelo da rede
n_nets = 30
N = nx.number_of_nodes(grafos[g]) #G

#ER networks
av_degree = vetores_caracteristicos[g][0] #k1
p = av_degree/(N-1)
for i in range(0, n_nets):
    GER = nx.gnp_random_graph(N, p, seed=None, directed=False)
    Gcc = sorted(nx.connected_components(GER), key=len, reverse=True)
    GER = GER.subgraph(Gcc[0])
    GER = nx.convert_node_labels_to_integers(GER, first_label=0)
    k1, k2, variance, av_cl, l, r = measures(GER)
    x = [k1, k2, variance, av_cl, l, r]
    X.append(x)
    y.append(0.0)

#WS networks
k = int(av_degree)
p = 0.1 #probability of rewiring
for i in range(0, n_nets):
    GWS = nx.watts_strogatz_graph(N, k, p, seed=None)
    Gcc = sorted(nx.connected_components(GWS), key=len, reverse=True)
    GWS = GWS.subgraph(Gcc[0])
    GWS = nx.convert_node_labels_to_integers(GWS, first_label=0)
    k1, k2, variance, av_cl, l, r = measures(GWS)
    x = [k1, k2, variance, av_cl, l, r]
    X.append(x)
    y.append(1.0)

# BA networks
m = int(av_degree/2)
for i in range(0, n_nets):
    GBA = nx.barabasi_albert_graph(N, m)
    Gcc = sorted(nx.connected_components(GBA), key=len, reverse=True)
    GBA = GBA.subgraph(Gcc[0])
    GBA = nx.convert_node_labels_to_integers(GBA, first_label=0)
    k1,k2,variance,av_cl,l,r = measures(GBA)
    x = [k1,k2,variance,av_cl,l,r]
    X.append(x)
    y.append(2.0)

X = np.array(X)
y = np.array(y)

In [214]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_net = np.array(vetores_caracteristicos[g])
X_net = X_net.reshape(1,len(X_net))
X_net = scaler.transform(X_net)

k = 5
model = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
model.fit(X, y) # predição no conjunto de teste

KNeighborsClassifier(metric='euclidean')

In [215]:
y_pred = model.predict(X_net)
print('Classe grafo', redes[g], ': ', cl[int(y_pred)])

Classe grafo yeast_transcription :  ER
